## Análise e Previsão de Produtividade de Culturas com Base em Clima e Tendências Técnicas

Objetivo: Investigar como fatores climáticos (chuva, temperatura média, extremos) e eventos técnicos (ex: introdução de sementes transgênicas ou novas práticas) influenciaram a produtividade das grandes culturas no Brasil nas últimas décadas.

Perguntas de negócio:

- Há uma correlação clara entre clima e produtividade?
- Quais culturas são mais sensíveis a extremos climáticos?
- Quando os saltos de produtividade ocorreram, houve relação com tecnologia ou clima favorável?

Fontes de dados:

- Produção agrícola por cultura e estado: IBGE – PAM
- Clima histórico: INMET (ou NOAA/ERA5 para dados globais diários)
- Eventos técnicos: você pode simular ou anotar marcos com base em artigos ou estudos (ex: “adoção de soja transgênica em 2005”)

Entregáveis:

- Dashboard exploratório com visualizações interativas por cultura, região, ano
- Análise descritiva e gráfica de correlações
- Mapa de calor ou gráfico de dispersão mostrando padrões regionais

In [1]:
import requests
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import geopandas as gpd
from ipywidgets import widgets
from IPython.display import display
import plotly.graph_objects as go


In [4]:
import geobr
# Gerar dados geograficos para mapas
municipalities = geobr.read_municipality()
municipalities=municipalities.rename({"name_muni": "Municipio",'abbrev_state':"Estado"}, axis=1)
municipalities['Muni_Estado'] = municipalities['Municipio'] + ' - ' + municipalities['Estado']

regions = geobr.read_meso_region()
regions['Mesorregião'] = regions['name_meso'] + ' - ' + regions['abbrev_state']
regions = gpd.GeoDataFrame(regions[['Mesorregião','geometry']])

# Para criar mapas no Power BI
'''
import topojson
regions_dissolved = regions.dissolve(by='Mesorregião').reset_index()
regions_topo = topojson.Topology(regions_dissolved, prequantize=False)
with open("regions.topojson", "w") as f:
    f.write(regions_topo.to_json())
'''

'\nimport topojson\nregions_dissolved = regions.dissolve(by=\'Mesorregião\').reset_index()\nregions_topo = topojson.Topology(regions_dissolved, prequantize=False)\nwith open("regions.topojson", "w") as f:\n    f.write(regions_topo.to_json())\n'

Coleta e processamento de dados a nível de mesorregião

In [ ]:
# Configurações
BASE_URL = 'https://apisidra.ibge.gov.br/values'
TABELA = '1612'  # Tabela de produção agrícola municipal
NIVEL = '8'      # Nível 8 = Mesorregião

# Produtos desejados (códigos da classificação 81)
PRODUTOS = ['2713', '2711', '2702', '2716', '2692']
PRODUTOS_NOMES = {
    '2713': 'Soja (em grão)',
    '2711': 'Milho (em grão)',
    '2702': 'Feijão (em grão)',
    '2716': 'Trigo (em grão)',
    '2692': 'Arroz (em casca)'
}

# Função para obter dados de uma variável e produto específico para todas as mesorregiões
def obter_dados(produto, ano):
    # Usando a mesma estrutura de URL que foi comprovada como funcional
    url = f"{BASE_URL}/t/{TABELA}/n{NIVEL}/all/p/{ano}/c81/{produto}/f/u"
    print(f"Obtendo dados: produto={produto}, ano={ano}")
    print(f"URL: {url}")
    
    try:
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if len(data) <= 1:  # Só tem o cabeçalho
                print(f"Sem dados para: produto={produto}, ano={ano}")
                return None
            
            # Converte para DataFrame
            df = pd.DataFrame(data[1:])  # Pula o cabeçalho
            
            print(f"Obtidos {len(df)} registros")
            return df
        else:
            print(f"Erro na requisição: {response.status_code}, {response.text}")
            return None
    except Exception as e:
        print(f"Erro ao obter dados: {str(e)}")
        return None


print("Iniciando coleta de dados da API do SIDRA/IBGE...")
# Lista para armazenar todos os dataframes
dataframes = []

# Para cada combinação de parâmetros
total_combinacoes = len(PRODUTOS) * (2023-1974)
combinacoes_processadas = 0

for ano in range(1974, 2023):
    for produto in PRODUTOS:
        # Adiciona um pequeno delay para evitar sobrecarregar a API
        time.sleep(0.3)
        
        df = obter_dados(produto, ano)
        if df is not None:
            dataframes.append(df)
        
        # Atualiza o progresso
        combinacoes_processadas += 1
        progresso = (combinacoes_processadas / total_combinacoes) * 100
        print(f"Progresso: {progresso:.1f}% ({combinacoes_processadas}/{total_combinacoes})")

# Verifica se obteve algum dado
if not dataframes:
    print("Nenhum dado foi obtido. Saindo.")

# Concatena todos os dataframes
print("Unindo todos os dados...")
resultado = pd.concat(dataframes, ignore_index=True)

# Salva o resultado em CSV
arquivo_saida = 'producao_agricola_mesorregiao.parquet'
resultado.to_parquet(arquivo_saida, index=False)
print(f"Dados salvos em: {arquivo_saida}")

print("Processamento concluído!")

Iniciando coleta de dados da API do SIDRA/IBGE...
Obtendo dados: produto=2713, ano=1974
URL: https://apisidra.ibge.gov.br/values/t/1612/n8/all/p/1974/c81/2713/f/u
Obtidos 685 registros
Progresso: 0.4% (1/245)
Obtendo dados: produto=2711, ano=1974
URL: https://apisidra.ibge.gov.br/values/t/1612/n8/all/p/1974/c81/2711/f/u
Obtidos 685 registros
Progresso: 0.8% (2/245)
Obtendo dados: produto=2702, ano=1974
URL: https://apisidra.ibge.gov.br/values/t/1612/n8/all/p/1974/c81/2702/f/u
Obtidos 685 registros
Progresso: 1.2% (3/245)
Obtendo dados: produto=2716, ano=1974
URL: https://apisidra.ibge.gov.br/values/t/1612/n8/all/p/1974/c81/2716/f/u
Obtidos 685 registros
Progresso: 1.6% (4/245)
Obtendo dados: produto=2692, ano=1974
URL: https://apisidra.ibge.gov.br/values/t/1612/n8/all/p/1974/c81/2692/f/u
Obtidos 685 registros
Progresso: 2.0% (5/245)
Obtendo dados: produto=2713, ano=1975
URL: https://apisidra.ibge.gov.br/values/t/1612/n8/all/p/1975/c81/2713/f/u
Obtidos 685 registros
Progresso: 2.4% (6/2

In [14]:
resultado = pd.read_parquet('producao_agricola_mesorregiao.parquet')

In [50]:
def processar_dados_mesorregiao_unificado(resultado, regions):
    """
    Processa dados agrícolas em nível de mesorregião em um único DataFrame consolidado.
    Elimina 'Área colhida' (pois é idêntico a Área Plantada), mantendo apenas os dados essenciais.
    """
    # Tipos que serão processados (removendo Área colhida)
    tipos_a_processar = ['Área plantada', 'Quantidade produzida', 'Rendimento médio da produção', 'Valor da produção']
    
    # Inicializar um dicionário para armazenar os DataFrames temporários
    dfs_temp = {}
    
    # Processar cada tipo de dado em um DataFrame temporário
    for tipo in tipos_a_processar:
        # Filtrar resultado pelo tipo atual
        df_temp = resultado[resultado['D4N'] == tipo].copy()
        
        # Converter e limpar valores
        df_temp.loc[:, 'V'] = df_temp['V'].replace(['...', '-', np.nan], 0)
        df_temp.loc[:, 'V'] = pd.to_numeric(df_temp['V'])
        df_temp.loc[:, 'D2N'] = pd.to_numeric(df_temp['D2N'])
        
        # Renomear colunas básicas
        df_temp = df_temp.rename(columns={
            'D2N': 'Ano',
            'D3N': 'Produto',
            'D1N': 'Mesorregião'
        })
        
        # Limpar nomes de produtos (remover texto entre parênteses)
        df_temp['Produto'] = df_temp['Produto'].apply(
            lambda x: x.split(' (')[0] if isinstance(x, str) and ' (' in x else x
        )
        
        # Definir o nome da coluna de valor baseado no tipo
        if tipo == 'Área plantada':
            novo_nome_valor = 'Area_Plantada_Hectares'
        elif tipo == 'Quantidade produzida':
            novo_nome_valor = 'Producao_Toneladas'
        elif tipo == 'Rendimento médio da produção':
            novo_nome_valor = 'Rendimento_KgPorHectare'
        elif tipo == 'Valor da produção':
            novo_nome_valor = 'Valor_Produzido_Mil_Reais'
        
        # Selecionar apenas as colunas necessárias
        df_temp = df_temp[['Mesorregião', 'Ano', 'Produto', 'V']]
        
        # Renomear a coluna de valor para o nome específico
        df_temp = df_temp.rename(columns={'V': novo_nome_valor})
        
        # Armazenar no dicionário
        dfs_temp[tipo] = df_temp
    
    # Mesclar os DataFrames usando as colunas de identificação como chaves
    # Começar com o primeiro DataFrame
    df_consolidado = dfs_temp[tipos_a_processar[0]]
    
    # Mesclar com os outros DataFrames
    for tipo in tipos_a_processar[1:]:
        df_consolidado = pd.merge(
            df_consolidado,
            dfs_temp[tipo],
            on=['Mesorregião', 'Ano', 'Produto'],
            how='outer'
        )
    
    # Preencher valores ausentes resultantes da mesclagem
    df_consolidado = df_consolidado.fillna(0)
    
    # Filtrar para manter apenas anos >= 1990 e com dados diferentes de 0
    df_consolidado = df_consolidado[df_consolidado['Ano'] >= 1990]
    df_consolidado = df_consolidado[~((df_consolidado['Area_Plantada_Hectares'] == 0) & (df_consolidado['Producao_Toneladas'] == 0) & (df_consolidado['Rendimento_KgPorHectare'] == 0))]
    
    # Remover os dados de Valor_Produzido_Reais de antes de 1994, pré-Real
    df_consolidado.loc[df_consolidado['Ano'] < 1994, 'Valor_Produzido_Mil_Reais'] = 0
    
    # Salvar o DataFrame consolidado
    df_consolidado.to_parquet('dados_agricolas_mesorregiao.parquet', index=False)
    
    return df_consolidado

# Exemplo de uso:
df_consolidado = processar_dados_mesorregiao_unificado(resultado, regions)

C:\Users\luiz_\AppData\Local\Temp\ipykernel_12272\3160104083.py:67: FutureWarning:

Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`



In [5]:
df_consolidado = pd.read_parquet('dados_agricolas_mesorregiao.parquet')
df_consolidado

,Mesorregião,Ano,Produto,Area_Plantada_Hectares,Producao_Toneladas,Rendimento_KgPorHectare,Valor_Produzido_Mil_Reais
0,Agreste Alagoano - AL,1990,Arroz,360,312,1300,0
1,Agreste Alagoano - AL,1990,Feijão,29622,15996,582,0
2,Agreste Alagoano - AL,1990,Milho,12566,5930,558,0
3,Agreste Alagoano - AL,1991,Arroz,547,1188,2171,0
4,Agreste Alagoano - AL,1991,Feijão,32113,16381,510,0
...,...,...,...,...,...,...,...
16843,Zona da Mata - MG,2021,Feijão,23014,19621,853,92044
16844,Zona da Mata - MG,2021,Milho,31687,140683,4440,242745
16845,Zona da Mata - MG,2022,Arroz,96,311,3240,442
16846,Zona da Mata - MG,2022,Feijão,23679,21027,888,109007


In [7]:
rendimento = df_consolidado[['Produto', 'Ano', 'Mesorregião', 'Rendimento_KgPorHectare']].copy()

# Create dropdown widgets only once and store them as global variables
region_dropdown = widgets.Dropdown(
    options=sorted(rendimento['Mesorregião'].unique()),
    description='Região:',
    style={'description_width': 'initial'}
)

rendimento = df_consolidado[['Produto', 'Ano', 'Mesorregião', 'Rendimento_KgPorHectare']].copy()

# Dicionário com anos e tipo de fenômeno
fenomenos_climaticos = {
    1990: 'El Niño',
    1991: 'El Niño',
    1992: 'El Niño',
    1993: 'Neutro',
    1994: 'El Niño',
    1995: 'La Niña',
    1996: 'La Niña',
    1997: 'El Niño',
    1998: 'La Niña',
    1999: 'La Niña',
    2000: 'La Niña',
    2001: 'Neutro',
    2002: 'El Niño',
    2003: 'Neutro',
    2004: 'El Niño',
    2005: 'Neutro',
    2006: 'El Niño',
    2007: 'La Niña',
    2008: 'La Niña',
    2009: 'El Niño',
    2010: 'La Niña',
    2011: 'La Niña',
    2012: 'La Niña',
    2013: 'Neutro',
    2014: 'Neutro',
    2015: 'El Niño',
    2016: 'El Niño',
    2017: 'Neutro',
    2018: 'El Niño',
    2019: 'Neutro',
    2020: 'La Niña',
    2021: 'La Niña',
    2022: 'La Niña'
}

# Cores para cada fenômeno
cores_fenomenos = {
    'El Niño': 'rgba(255, 100, 100, 0.2)',  # vermelho claro
    'La Niña': 'rgba(100, 100, 255, 0.2)',  # azul claro
    'Neutro': 'rgba(200, 200, 200, 0.2)'    # cinza claro
}

def update_plot(region):
    filtered_data = rendimento[rendimento['Mesorregião'] == region]

    fig = go.Figure()

    for produto in filtered_data['Produto'].unique():
        df_produto = filtered_data[filtered_data['Produto'] == produto]
        fig.add_trace(go.Scatter(
            x=df_produto['Ano'],
            y=df_produto['Rendimento_KgPorHectare'],
            mode='lines+markers',
            name=produto
        ))

    # Adiciona faixas coloridas no fundo para El Niño / La Niña / Neutro
    for ano, fenomeno in fenomenos_climaticos.items():
        fig.add_shape(
            type='rect',
            x0=ano - 0.5,  # início do ano
            x1=ano + 0.5,  # fim do ano
            y0=0,
            y1=1,
            xref='x',
            yref='paper',
            fillcolor=cores_fenomenos.get(fenomeno, 'rgba(0,0,0,0)'),
            layer='below',
            line_width=0
        )

    fig.update_layout(
        title=f'Rendimento anual em {region}',
        xaxis_title='Ano',
        yaxis_title='Rendimento (Kg/ha)',
        yaxis=dict(rangemode='tozero'),
        legend_title='Produto'
    )

    fig.show()

# Create the interactive plot without creating new dropdowns
interactive_plot = widgets.interactive(update_plot, region=region_dropdown)

# Display the interactive plot
display(interactive_plot)


interactive(children=(Dropdown(description='Região:', options=('Agreste Alagoano - AL', 'Agreste Paraibano - P…

In [8]:
# Group by Year and Product first, then get top 5 producers for each combination
ranking = df_consolidado[['Produto', 'Ano', 'Mesorregião', 'Producao_Toneladas']].copy()
ranking = ranking.rename(columns={'Producao_Toneladas': 'Produção'})

# Sort within each group and get top 5
ranking = (ranking.groupby(['Ano', 'Produto'])
          .apply(lambda x: x.nlargest(5, 'Produção'))
          .reset_index(drop=True))

# Sort by Product, Year for better visualization
ranking = ranking.sort_values(['Produto', 'Ano', 'Produção'], ascending=[True, True, False])
ranking = ranking.reset_index(drop=True)
ranking


C:\Users\luiz_\AppData\Local\Temp\ipykernel_14312\4081464389.py:7: DeprecationWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.



,Produto,Ano,Mesorregião,Produção
0,Arroz,1990,Sudoeste Rio-grandense - RS,1066281
1,Arroz,1990,Sudeste Rio-grandense - RS,801426
2,Arroz,1990,Metropolitana de Porto Alegre - RS,656604
3,Arroz,1990,Centro Ocidental Rio-grandense - RS,356794
4,Arroz,1990,Sul Catarinense - SC,252561
...,...,...,...,...
820,Trigo,2022,Noroeste Rio-grandense - RS,4012389
821,Trigo,2022,Centro Oriental Paranaense - PR,609435
822,Trigo,2022,Sudoeste Paranaense - PR,564785
823,Trigo,2022,Norte Central Paranaense - PR,555813


In [26]:
rendimento_medio = rendimento.groupby(['Produto', 'Ano']).agg({'Rendimento_KgPorHectare': 'mean'}).reset_index()

def update_plot():
    fig = go.Figure()

    # Add lines for each product
    for produto in rendimento_medio['Produto'].unique():
        df_produto = rendimento_medio[rendimento_medio['Produto'] == produto]
        fig.add_trace(go.Scatter(
            x=df_produto['Ano'],
            y=df_produto['Rendimento_KgPorHectare'],
            mode='lines+markers',
            name=produto,
            line=dict(width=2),
            marker=dict(size=6)
        ))

    # Update layout with better formatting
    fig.update_layout(
        title=dict(
            text='Rendimento Médio das Culturas (1990-2022)',
            x=0.5,
            font=dict(size=20)
        ),
        xaxis=dict(
            title='Ano',
            gridcolor='lightgray',
            showgrid=True
        ),
        yaxis=dict(
            title='Rendimento (Kg/ha)',
            gridcolor='lightgray',
            showgrid=True,
            rangemode='tozero'
        ),
        legend=dict(
            title='Culturas',
            yanchor="top",
            y=0.99,
            xanchor="left",
            x=1.02
        ),
        showlegend=True,
        plot_bgcolor='white',
        hovermode='x unified',
        width=1000,
        height=600
    )

    fig.show()

# Create and display the interactive plot
interactive_plot = widgets.interactive(update_plot)
display(interactive_plot)


interactive(children=(Output(),), _dom_classes=('widget-interact',))

In [30]:
df_consolidado[['Area_Plantada_Hectares', 'Rendimento_KgPorHectare', 'Valor_Produzido_Mil_Reais']].corr()

,Area_Plantada_Hectares,Rendimento_KgPorHectare,Valor_Produzido_Mil_Reais
Area_Plantada_Hectares,1.000000,0.158059,0.757416
Rendimento_KgPorHectare,0.158059,1.000000,0.161900
Valor_Produzido_Mil_Reais,0.757416,0.161900,1.000000


In [31]:
#   - Coeficiente de variação do rendimento por cultura e região
df_consolidado[['Produto', 'Mesorregião', 'Rendimento_KgPorHectare']].groupby(['Produto', 'Mesorregião']).agg(
    Rendimento_Medio=('Rendimento_KgPorHectare', 'mean'),
    Rendimento_Desvio=('Rendimento_KgPorHectare', 'std')
).reset_index().assign(
    Coeficiente_Variacao=lambda x: x['Rendimento_Desvio'] / x['Rendimento_Medio'] * 100
).sort_values(['Produto', 'Coeficiente_Variacao'], ascending=[True, False]).reset_index(drop=True)

,Produto,Mesorregião,Rendimento_Medio,Rendimento_Desvio,Coeficiente_Variacao
0,Arroz,Borborema - PB,909.777778,840.968367,92.436679
1,Arroz,Mata Paraibana - PB,3250.600000,2273.490474,69.940641
2,Arroz,Sertão Pernambucano - PE,1909.575758,1310.670488,68.636737
3,Arroz,Marajó - PA,2439.787879,1650.782575,67.660906
4,Arroz,Norte Goiano - GO,2630.212121,1751.407590,66.588074
...,...,...,...,...,...
563,Trigo,Norte de Minas - MG,674.000000,22.516660,3.340751
564,Trigo,Noroeste Goiano - GO,4866.666667,115.470054,2.372672
565,Trigo,Araraquara - SP,0.000000,NaN,NaN
566,Trigo,Centro-Sul Mato-grossense - MT,2000.000000,NaN,NaN


In [ ]:
meteo = pd.read_csv('meteo.csv.gz', compression='gzip')

cod_muni = pd.read_csv('cod_muni.csv.gz', compression='gzip')
cod_muni['Mesorregião'] = cod_muni['nome_mesorregiao'] + ' - ' + cod_muni['sigla_uf']
cod_muni=cod_muni[['id_municipio', 'Mesorregião']]

meteo=meteo.merge(cod_muni, on='id_municipio', how='left')
meteo = meteo[['id_estacao', "Mesorregião"]]

Dados meteorológicos baixados do BDMEP através do BigQuery com a seguinte query para já agregar os dados (originalmente em hora) e reduzir o tamanho do arquivo


    SELECT
    ano,
    id_estacao,
    SUM(precipitacao_total) AS precipitacao_total_anual,
    AVG(radiacao_global) AS radiacao_global_media,
    AVG(temperatura_bulbo_hora) AS temperatura_bulbo_media,
    AVG(vento_velocidade) AS vento_velocidade_media

    FROM
      basedosdados.br_inmet_bdmep.microdados

    GROUP BY
      ano, id_estacao

    ORDER BY
      ano, id_estacao


In [79]:
dados_meteo = pd.read_csv('dados_meteo.csv').rename(columns={'ano': 'Ano'})
dados_meteo = dados_meteo.merge(meteo, on='id_estacao', how='left').drop(columns=['id_estacao'])


,Ano,precipitacao_total_anual,radiacao_global_media,temperatura_bulbo_media,vento_velocidade_media,Mesorregião
0,2000,656.0,1531.114109,20.925588,2.572984,Distrito Federal - DF
1,2000,738.4,1529.583786,26.734560,1.102612,Centro Amazonense - AM
2,2000,172.2,1654.830123,25.895893,2.070673,Metropolitana de Salvador - BA
3,2000,524.2,1257.407504,22.580746,2.335747,Metropolitana do Rio de Janeiro - RJ
4,2000,401.8,1410.367491,20.856789,2.087978,Metropolitana de Porto Alegre - RS
...,...,...,...,...,...,...
9454,2024,982.4,1291.119353,22.462063,3.247565,Centro Ocidental Paranaense - PR
9455,2024,1195.8,688.917304,20.669404,3.073476,Centro-Sul Paranaense - PR
9456,2024,854.2,1176.623194,18.081684,1.294182,Metropolitana de Curitiba - PR
9457,2024,1621.4,560.782117,19.430490,2.392655,NaN


In [80]:
df_meteo = df_consolidado.merge(dados_meteo, on=['Mesorregião', 'Ano'], how='left')

In [82]:
df_meteo.columns

Index(['Mesorregião', 'Ano', 'Produto', 'Area_Plantada_Hectares',
       'Producao_Toneladas', 'Rendimento_KgPorHectare',
       'Valor_Produzido_Mil_Reais', 'precipitacao_total_anual',
       'radiacao_global_media', 'temperatura_bulbo_media',
       'vento_velocidade_media'],
      dtype='object')

In [89]:
df_meteo.corr(numeric_only=True)
df_meteo.to_parquet('dados_meteo.parquet', index=False)

Análises a se incluir no dashboard:

1. **Análise de tendências temporais**:
   - Gráficos de linha mostrando a evolução do rendimento médio de cada cultura ao longo do tempo
   - Identificação de pontos de inflexão (anos com mudanças significativas)

2. **Comparativos regionais**:
   - Ranking das mesorregiões mais produtivas para cada cultura
   - Mapas de calor mostrando a distribuição espacial da produtividade

3. **Correlações entre variáveis**:
   - Relação entre área plantada e rendimento (verificar se há economias de escala)
   - Correlação entre valor da produção e rendimento

4. **Análise de volatilidade**:
   - Coeficiente de variação do rendimento por cultura e região
   - Identificação das regiões e culturas mais estáveis/instáveis

5. **Taxonomia de mesorregiões**:
   - Agrupamento de regiões com padrões similares de produtividade
   - Classificação por perfil de culturas predominantes

6. **Séries temporais avançadas**:
   - Decomposição das séries (tendência, sazonalidade, resíduos)
   - Detecção de outliers e eventos extremos

7. **Indicadores de especialização regional**:
   - Índice de concentração para identificar especialização por cultura
   - Evolução da diversificação agrícola nas mesorregiões

8. **Visualizações interativas**:
   - Filtros por cultura, período e região
   - Animações mostrando mudanças espaciais ao longo do tempo

Estas análises fornecerão uma base sólida para, posteriormente, integrar os dados climáticos e identificar correlações com os padrões de produtividade.

In [2]:
df = pd.read_parquet('dados_meteo.parquet')